In [1]:
#Include all the libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from statistics import mean 

import string 

#Sklearn Libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


#Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# 1st Part

In [2]:
train_txt = pd.read_csv("train.csv").fillna('unknown')


In [3]:
#cleaning the not needed columns
train_txt = train_txt.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1)
train_txt.head()
train_txt.shape

(159571, 3)

In [5]:
#splitting training and testing
X = train_txt.comment_text

y = train_txt[['toxic']]
X_train, X_val, y_train, y_val = train_test_split(X,y,shuffle=True, test_size=0.1, random_state= 42)

In [ ]:
#print to check the X_train and X_val set
X_train.shape, X_val.shape

In [6]:
#stop word removal and cleaning 

stop_words = _stop_words.ENGLISH_STOP_WORDS
def clean(doc):
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stop_words])
    return doc.lower()

In [7]:
# making Bag of words
vect = CountVectorizer(max_features=4000,preprocessor=clean)
print(vect)
X_train_vec = vect.fit_transform(X)
X_val_vec = vect.transform(y)
print(X_train_vec)

CountVectorizer(max_features=4000,
                preprocessor=<function clean at 0x7ff6cc5be940>)
  (0, 1261)	1
  (0, 3887)	1
  (0, 1122)	1
  (0, 3739)	1
  (0, 1311)	1
  (0, 3010)	1
  (0, 3546)	1
  (0, 3869)	1
  (0, 1916)	1
  (0, 1473)	1
  (0, 3811)	1
  (0, 2313)	1
  (0, 3983)	1
  (0, 1279)	1
  (0, 156)	1
  (0, 1066)	1
  (0, 2933)	1
  (0, 3509)	1
  (0, 3481)	1
  (0, 2480)	1
  (0, 1712)	1
  (0, 3000)	1
  (1, 3481)	1
  (1, 1712)	1
  (1, 1587)	1
  :	:
  (159569, 944)	1
  (159569, 3313)	1
  (159569, 2072)	1
  (159569, 2026)	1
  (159569, 3778)	1
  (159569, 48)	1
  (159569, 2075)	1
  (159570, 156)	1
  (159570, 1066)	1
  (159570, 2843)	1
  (159570, 1793)	1
  (159570, 3552)	1
  (159570, 3873)	1
  (159570, 491)	1
  (159570, 312)	2
  (159570, 3682)	1
  (159570, 1508)	1
  (159570, 3032)	1
  (159570, 683)	1
  (159570, 303)	2
  (159570, 1934)	1
  (159570, 1691)	1
  (159570, 1693)	1
  (159570, 3022)	1
  (159570, 1610)	1


In [ ]:
type(X_val_vec)

In [ ]:
print(X_train_vec.shape,X_val_vec.shape)

In [ ]:
nb = MultiOutputClassifier(MultinomialNB()).fit(X_train_vec,y)

In [ ]:
y_pred = nb.predict(X_val_vec)

In [ ]:
#accuracy = accuracy_score(y_val,y_pred)
#print("Accuracy:",accuracy)

In [ ]:
print(type(y_val))
print(type(y_pred))

# 2nd Part

In [ ]:

test_labels = pd.read_csv("test_labels.csv")
test_labels = test_labels[test_labels['toxic'].isin([0, 1])]

test_label_id = test_labels.id
test_labels.head()

test_txt = pd.read_csv("test.csv").fillna('unknown')
final_test = test_txt.loc[test_txt['id'].isin(test_label_id)]
final_test.head()

In [ ]:
df_test_vec = vect.transform(final_test['comment_text'])
print(type(df_test_vec))
test_pred = nb.predict(df_test_vec)


In [ ]:
true_label = test_labels['toxic'].values
accuracy = accuracy_score(test_pred,true_label)
print(accuracy)

In [ ]:
# Calculate precision
precision = precision_score(test_pred,true_label)

# Calculate recall
recall = recall_score(test_pred,true_label)

# Calculate F1-score
f1 = f1_score(test_pred,true_label)

# Calculate ROC AUC score
roc_auc = roc_auc_score(test_pred,true_label)
print(precision)
print(recall)
print(f1)
print(roc_auc)

In [ ]:
test_prob = nb.predict_proba(df_test_vec)
#print(test_prob[0][2][1])
toxic_prob_list = []
for i in range(len(test_prob[0])):
    toxic_prob = test_prob[0][i][1]
    toxic_prob_list.append(toxic_prob)

print(type(toxic_prob_list))

In [ ]:
#input the file into new dataframe
test_txt['prediction'] = test_pred
test_txt['Probability'] = toxic_prob_list
test_txt.head()

In [ ]:
test_txt.to_csv('Output.csv')